# Homework 1 - Streaming (15 pts)

### DUE: 02/18/2020 at 5:30pm

This homework is for practicing Python’s *generators* and the design of *streaming* algorithms in general. We’re going to use the **taxi_events.csv** and **taxi_trips.csv** data sets. Both of them are available on NYU Classes under *Data Sets* section. You are required to turn in this notebook with all the parts filled in place of <###>. Your notebook must be named BDM\_HW1\_NetID.ipynb.

## Task 1 (8 points)

Your task is to **compute the maximum number of active taxi cabs** that were hired at any point in time. This the same as computing the maximum number of taxi cabs that have passengers. The input data set is **taxi_events.csv**, which logged all pick-up and drop-off events for all taxi trips. The description of the fields in this file is as follows:

|Column name|Description|
|--|--|
|time |The timestamp of the event. All events are sorted increasingly by their timestamps. |
|vehicle_id |The unique ID of the taxi vehicle involved in this event. |
|event |A string of either *"pickup"* or *"dropoff"* for describing the drop-off or pick-up event, respectively. |

For example, let's assume that on *Feb-01-2015*, there was a taxi that picked up a passenger at midnight and dropped her off at 14 minutes past midnight. If the taxi cab has the Vehicle ID of *V102*, then you should see two events being logged in this data set as:

<pre>
...
2015-02-01 00:00:00+00,V102,pickup
...
2015-02-01 00:14:00+00,V102,dropoff
...
</pre>

You are given the above data set in a streaming fashion (reading in row by row), and must design a streaming algorithm that uses the least possible additional memory to compute the maximum number of hired taxi cabs at any point in time. Again, this is equivalent to having a virtual dispatcher, who repeatedly ask every second *"how many taxis are being hired (having passengers) at the moment?"*, and then log the maximum number during the entire period.

Please modify the code snippet below to complete this task. Your code should only output a single number, which is the maximum number of hired taxi cabs. Of course, you can add additional initialization codes outside of the for loop as needed. Additional, please provide a brief rationale and/or justification for your design after the code.

In [17]:
import csv

def csvRows(filename):
    with open(filename, 'r') as fi:
        reader = csv.DictReader(fi)
        for row in reader:
            yield row

maxHiredCabs = 0
currentHiredCabs = 0
cab = []
for row in csvRows('taxi_events.csv'):
    if row['event'] == 'pickup' and row['vehicle_id'] not in cab:
        currentHiredCabs += 1
        cab.append(row['vehicle_id'])
    elif row['event'] == 'dropoff' and row['vehicle_id'] in cab:
        currentHiredCabs -= 1
        cab.remove(row['vehicle_id'])
        
    if currentHiredCabs > maxHiredCabs:
        maxHiredCabs = currentHiredCabs
    
print(maxHiredCabs)

250


#### RATIONALE AND JUSTIFICATION

Define *maxHireCabs* to collect the maximum number of hired taxi cabs during the entire time, *currentHiredCabs* to collect the current number of hired taxi cabs till the time of each record, and *cab* to collect the vehicle id of the current hired taxi cabs to prevent the situation that the same cab pick up multiple passengers during this time and be counted several times.

For every item in this data, if it's a 'pickup' event and the cab has not been recorded before, then there's one more current hired cab, and we collect its vehicle id in *cab*; if it's a 'dropoff' event and the cab exists in the *cab* list, then there's one less cab in service, and we remove its vehicle id from the *cab* list.

Then we check if the number of current hired cabs is the maximum number until this time, and then go through the loop again. And finally, print out *maxHiredCabs* as the result.

## Task 2 (7 points)

The objective of this task is identical to Task 1's but you are asked to use the full **taxi_trips.csv** data set instead of the events. The main difference (and challenge) is that both pick-up and drop-off event for each trip is now presented in a single record, thus, the drop-off events are not sorted by their timestamps. You are again asked to do this in a streaming fashion that needs to minimize the amount of memory usage. Please modify the code below accordingly, and also with a brief explaination of the solution.

Below is the description of the **taxi_trips.csv** file, which is sorted only by the pick-up time:

|Column name|Description|
|--|--|
|trip_duration |The duration of the trip in seconds. This field is for your convenience since it can be derived also from the pick-up and drop-off times. |
|pickup_time |The timestamp of the pick-up of the trip. All trip records are sorted increasingly by their pick-up times. |
|dropoff_time |The timestamp of the drop-off event. |
|vehicle_id |The unique ID of the taxi vehicle involved in this trip record. |



In [47]:
import csv

def csvRows(filename):
    with open(filename, 'r') as fi:
        reader = csv.DictReader(fi)
        for row in reader:
            yield row

maxHiredCabs = 0
droptime = []
cab = []
for row in csvRows('taxi_trips.csv'):
    
    if row['vehicle_id'] in cab and row['dropoff_time'] > droptime[cab.index(row['vehicle_id'])]:
        droptime[cab.index(row['vehicle_id'])] = row['dropoff_time']
        
    elif row['vehicle_id'] not in cab:
        #sort current cab list by drop-off time
        drop = dict(sorted(dict(zip(cab, droptime)).items(), key=lambda x: x[1]))
        droptime = list(drop.values())
        cab = list(drop.keys())
        
        #identify and remove the cabs that are no longer in service
        delta = len(list(filter(lambda x: x <= row['pickup_time'], droptime)))
        droptime = droptime[delta:]
        cab = cab[delta:]
        
        #update the cab list with drop-off time
        cab.append(row['vehicle_id'])
        droptime.append(row['dropoff_time'])
    
    if len(cab) > maxHiredCabs:
        maxHiredCabs = len(cab)
        
print(maxHiredCabs)

250


#### RATIONALE AND JUSTIFICATION

Define *maxHireCabs* to collect the maximum number of hired taxi cabs during the entire time, *droptime* to collect the time to drop off for current hired taxi cabs, and *cab* to collect the vehicle id of the current hired taxi cabs to prevent the situation that the same cab pick up multiple passengers during this time and be counted several times.

For every item in this data, if the cab is already in the list of cabs in service, we update the drop-off time if the new drop-off time is later; if the cab isn't in *cab*, we sort *droptime* and corresponding *cab* by drop-off time, and then find the cabs with drop-off time earlier or equal to the current time, an remove those cabs that no longer in service with its drop-off time information from *cab* and *droptime*, then we update *cab* and *droptime* with new records.

Then we check if the number of current hired cabs is the maximum number until this time, and then go through the loop again. And finally, print out *maxHiredCabs* as the result.